In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


city_data = pd.read_csv("data/Cities.csv")
conferences = pd.read_csv("data/Conferences.csv")
tourneygames = pd.read_csv("data/MConferenceTourneyGames.csv")
# tourneyresults = pd.read_csv("data/MNCAATourneyCompactResults.csv")
tourneydetailedresults = pd.read_csv("data/MNCAATourneyDetailedResults.csv")
tourneyseeds = pd.read_csv("data/MNCAATourneySeeds.csv")
# tourneyseedresutls = pd.read_csv("data/MNCAATourneySeedRoundSlots.csv")
regularseasoncompact = pd.read_csv("data/MRegularSeasonCompactResults.csv")
regularseasondetailed = pd.read_csv("data/MRegularSeasonDetailedResults.csv")
# seasons = pd.read_csv("data/MSeasons.csv")
teams = pd.read_csv("data/MTeams.csv")
# secondarytourneycompact = pd.read_csv("data/MSecondaryTourneyCompactResults.csv")
secondarytourneydetailed = pd.read_csv("data/MSecondaryTourneyTeams.csv")
# team_conerences = pd.read_csv("data/MTeamConferences.csv")
# team_spellings = pd.read_csv("data/MTeamSpellings.csv")
sample_submission = pd.read_csv("data/SampleSubmission2023.csv")

In [75]:
def win_loss(team, year, dataset):
    team_stats = dataset.loc[(dataset["WTeamID"] == team) & (dataset["Season"] == year)]
    team_stats = team_stats.append(dataset.loc[(dataset["LTeamID"] == team) & (dataset["Season"] == year)])
    wins = team_stats.loc[(team_stats["WTeamID"] == team)]
    losses = team_stats.loc[(team_stats["LTeamID"] == team)]
    return wins["WTeamID"].count() / (wins["WTeamID"].count() + losses["LTeamID"].count())

In [70]:
def offensive_rebound_rate(team, year, dataset):
    wins = dataset.loc[(dataset["WTeamID"] == team) & (dataset["Season"] == year)]
    losses = dataset.loc[(dataset["LTeamID"] == team) & (dataset["Season"] == year)]
    return (wins["WOR"].sum() + losses["LOR"].sum()) / (wins["WFGA"].sum() + losses["LFGA"].sum())

In [71]:
import re
def seed(team, year):
    try:
        seedgrab = tourneyseeds.loc[(tourneyseeds["TeamID"] == team) & (tourneyseeds["Season"] == year)]
        seedgrab = seedgrab[["Seed"]].values[0][0]
        seedgrab = str(seedgrab)
        print(seedgrab)
    except:
        return 18
    return int(re.search(r'\d+', seedgrab).group())

In [72]:
def offensive_rating(team, year, dataset):
    totalpoints = 0
    totalpossessions = 0
    team_stats = dataset.loc[(dataset["WTeamID"] == team) & (dataset["Season"] == year)]
    team_stats = team_stats.append(dataset.loc[(dataset["LTeamID"] == team) & (dataset["Season"] == year)])
    wins = team_stats.loc[(team_stats["WTeamID"] == team)]
    losses = team_stats.loc[(team_stats["LTeamID"] == team)]
    totalpoints += wins["WScore"].sum()
    totalpoints += losses["LScore"].sum()
    totalpossessions += wins["WFGA"].sum()
    totalpossessions += wins["WFGM3"].sum()
    totalpossessions += wins["WFTA"].sum()
    totalpossessions += losses["LFGA"].sum()
    totalpossessions += losses["LFGM3"].sum()
    totalpossessions += losses["LFTA"].sum()
    return totalpoints / totalpossessions

In [73]:
def defensive_rating(team, year, dataset):
    totalpoints = 0
    totalpossessions = 0
    team_stats = dataset.loc[(dataset["WTeamID"] == team) & (dataset["Season"] == year)]
    team_stats = team_stats.append(dataset.loc[(dataset["LTeamID"] == team) & (dataset["Season"] == year)])
    wins = team_stats.loc[(team_stats["WTeamID"] == team)]
    losses = team_stats.loc[(team_stats["LTeamID"] == team)]
    totalpoints += wins["LScore"].sum()
    totalpoints += losses["WScore"].sum()
    totalpossessions += wins["LFGA"].sum()
    totalpossessions += wins["LFGM3"].sum()
    totalpossessions += wins["LFTA"].sum()
    totalpossessions += losses["WFGA"].sum()
    totalpossessions += losses["WFGM3"].sum()
    totalpossessions += losses["WFTA"].sum()
    return totalpoints / totalpossessions

In [76]:
%%capture
regulardata = []
for year in range(2003, 2020):
    for team in range(1101, 1478):
        wins = win_loss(team, year, regularseasondetailed)
        offensive_rebound_rate_ = offensive_rebound_rate(team, year, regularseasondetailed)
        seed_ = seed(team, year)
        offensive_rating_ = offensive_rating(team, year, regularseasondetailed)
        defensive_rating_ = defensive_rating(team, year, regularseasondetailed)
        regulardata.append([year, team, wins, offensive_rebound_rate_, seed_, offensive_rating_, defensive_rating_])
for year in range(2021, 2024):
    for team in range(1101, 1478):
        wins = win_loss(team, year, regularseasondetailed)
        offensive_rebound_rate_ = offensive_rebound_rate(team, year, regularseasondetailed)
        seed_ = seed(team, year)
        offensive_rating_ = offensive_rating(team, year, regularseasondetailed)
        defensive_rating_ = defensive_rating(team, year, regularseasondetailed)
        regulardata.append([year, team, wins, offensive_rebound_rate_, seed_, offensive_rating_, defensive_rating_])

tourneydata = []

for year in range(2003, 2020):
    for team in range(1101, 1478):
        wins = win_loss(team, year, tourneydetailedresults)
        offensive_rebound_rate_ = offensive_rebound_rate(team, year, tourneydetailedresults)
        seed_ = seed(team, year)
        offensive_rating_ = offensive_rating(team, year, tourneydetailedresults)
        defensive_rating_ = defensive_rating(team, year, tourneydetailedresults)
        tourneydata.append([year, team, wins, offensive_rebound_rate_, seed_, offensive_rating_, defensive_rating_])

for year in range(2021, 2024):
    for team in range(1101, 1478):
        wins = win_loss(team, year, tourneydetailedresults)
        offensive_rebound_rate_ = offensive_rebound_rate(team, year, tourneydetailedresults)
        seed_ = seed(team, year)
        offensive_rating_ = offensive_rating(team, year, tourneydetailedresults)
        defensive_rating_ = defensive_rating(team, year, tourneydetailedresults)
        tourneydata.append([year, team, wins, offensive_rebound_rate_, seed_, offensive_rating_, defensive_rating_])
inputdata1 = pd.DataFrame(regulardata, columns=["Year", "TeamID", "Wins", "OffensiveReboundRate", "Seed", "OffensiveRating", "DefensiveRating"])
inputdata2 = pd.DataFrame(tourneydata, columns=["Year", "TeamID", "Wins", "OffensiveReboundRate", "Seed", "OffensiveRating", "DefensiveRating"])
;

In [78]:
inputdata1

,Year,TeamID,Wins,OffensiveReboundRate,Seed,OffensiveRating,DefensiveRating
0,2003,1101,NaN,NaN,18,NaN,NaN
1,2003,1102,0.428571,0.105027,18,0.884658,0.858065
2,2003,1103,0.481481,0.175066,18,0.903952,0.910660
3,2003,1104,0.607143,0.237352,10,0.820296,0.822785
4,2003,1105,0.269231,0.219101,18,0.788340,0.854265
...,...,...,...,...,...,...,...
7535,2023,1473,0.275862,0.119421,18,0.837676,0.867138
7536,2023,1474,0.500000,0.157240,18,0.848859,0.898137
7537,2023,1475,0.448276,0.168729,18,0.830438,0.858796
7538,2023,1476,0.433333,0.094723,18,0.860738,0.846493


In [134]:
import random
x_train_reg = pd.DataFrame()
y_train_reg = []
teamid = pd.DataFrame()
for year in range(2003, 2020):
    for game in range(len(regularseasondetailed.loc[(regularseasondetailed["Season"] == year)])):
        ranseed = random.randint(0, 1)
        first = ""
        sec = ""
        if ranseed == 0:
            first = "WTeamID"
            sec = "LTeamID"
        else:
            first = "LTeamID"
            sec = "WTeamID"
        team1 = inputdata1.loc[(inputdata1["Year"] == year) & (inputdata1["TeamID"] == regularseasondetailed[first][game])]
        team2 = inputdata1.loc[(inputdata1["Year"] == year) & (inputdata1["TeamID"] == regularseasondetailed[sec][game])]
        team1 = team1.drop(["Year", "TeamID"], axis=1)
        team2 = team2.drop(["Year", "TeamID"], axis=1)
        team1.reset_index(drop=True, inplace=True)
        team2.reset_index(drop=True, inplace=True)
        team1 = team1.subtract(team2, axis="columns")
        x_train_reg = pd.concat([x_train_reg, team1], axis=0, ignore_index=True)
        try:
            y_train_reg.append(ranseed)
        except:
            y_train_reg = [ranseed]
for year in range(2003, 2020):
    for game in range(len(tourneydetailedresults.loc[(tourneydetailedresults["Season"] == year)])):
        ranseed = random.randint(0, 1)
        first = ""
        sec = ""
        if ranseed == 0:
            first = "WTeamID"
            sec = "LTeamID"
        else:
            first = "LTeamID"
            sec = "WTeamID"
        team1 = inputdata1.loc[(inputdata1["Year"] == year) & (inputdata1["TeamID"] == tourneydetailedresults[first][game])]
        team2 = inputdata1.loc[(inputdata1["Year"] == year) & (inputdata1["TeamID"] == tourneydetailedresults[sec][game])]
        team1 = team1.drop(["Year", "TeamID"], axis=1)
        team2 = team2.drop(["Year", "TeamID"], axis=1)
        team1.reset_index(drop=True, inplace=True)
        team2.reset_index(drop=True, inplace=True)
        team1 = team1.subtract(team2, axis="columns")
        x_train_reg = pd.concat([x_train_reg, team1], axis=0, ignore_index=True)
        y_train_reg.append(ranseed)
x_valid = pd.DataFrame()
y_valid = []
for year in [ 2021, 2022]:
    for game in range(len(regularseasondetailed.loc[(regularseasondetailed["Season"] == year)])):
        ranseed = random.randint(0, 1)
        first = ""
        sec = ""
        if ranseed == 0:
            first = "WTeamID"
            sec = "LTeamID"
        else:
            first = "LTeamID"
            sec = "WTeamID"
        team1 = inputdata1.loc[(inputdata1["Year"] == year) & (inputdata1["TeamID"] == regularseasondetailed[first][game])]
        team2 = inputdata1.loc[(inputdata1["Year"] == year) & (inputdata1["TeamID"] == regularseasondetailed[sec][game])]
        team1 = team1.drop(["Year", "TeamID"], axis=1)
        team2 = team2.drop(["Year", "TeamID"], axis=1)
        team1.reset_index(drop=True, inplace=True)
        team2.reset_index(drop=True, inplace=True)
        team1 = team1.subtract(team2, axis="columns")
        x_valid = pd.concat([x_valid, team1], axis=0, ignore_index=True)
        try:
            y_valid.append(ranseed)
        except:
            y_valid = [ranseed]
for year in [2021, 2022]:
    for game in range(len(tourneydetailedresults.loc[(tourneydetailedresults["Season"] == year)])):
        ranseed = random.randint(0, 1)
        first = ""
        sec = ""
        if ranseed == 0:
            first = "WTeamID"
            sec = "LTeamID"
        else:
            first = "LTeamID"
            sec = "WTeamID"
        team1 = inputdata1.loc[(inputdata1["Year"] == year) & (inputdata1["TeamID"] == tourneydetailedresults[first][game])]
        team2 = inputdata1.loc[(inputdata1["Year"] == year) & (inputdata1["TeamID"] == tourneydetailedresults[sec][game])]
        team1 = team1.drop(["Year", "TeamID"], axis=1)
        team2 = team2.drop(["Year", "TeamID"], axis=1)
        team1.reset_index(drop=True, inplace=True)
        team2.reset_index(drop=True, inplace=True)
        team1 = team1.subtract(team2, axis="columns")
        x_valid = pd.concat([x_valid, team1], axis=0, ignore_index=True)
        y_valid.append(ranseed)

In [135]:
x_train_reg

,Wins,OffensiveReboundRate,Seed,OffensiveRating,DefensiveRating
0,0.192857,-0.022729,-9,0.041286,-0.035606
1,-0.034483,0.004438,4,-0.050669,0.006404
2,-0.321429,0.015855,15,-0.094892,0.023468
3,0.094470,-0.031152,0,-0.034124,-0.042453
4,-0.082011,-0.050594,17,0.025535,0.047359
...,...,...,...,...,...
88614,-0.106061,-0.020274,10,0.026448,0.037078
88615,0.129234,-0.016014,-4,0.057753,0.008681
88616,0.300334,-0.046345,0,0.099067,0.020379
88617,-0.014205,-0.026395,0,0.046802,-0.008743


In [136]:
y_train_reg = pd.DataFrame(y_train_reg)
y_valid = pd.DataFrame(y_valid)

In [137]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
x_train_cleaned = min_max_scaler.fit_transform(x_train_reg)
x_valid = min_max_scaler.fit_transform(x_valid)
x_train_cleaned = tf.convert_to_tensor(x_train_cleaned)
y_train_cleaned = tf.convert_to_tensor(y_train_reg)

In [141]:
all_inputs = []
encoded_features = []
for i in ["Wins", "ORR", "SDF", "OFR", "DFR"]:
    if i == "SDF":
        seed_col = tf.keras.Input(shape=(1,), name='SDF', dtype='int64')
        encoding_layer = tf.keras.layers.Embedding(17, 1, input_length=1)(seed_col)
        encoding_layer = tf.keras.layers.Flatten()(encoding_layer)
        all_inputs.append(seed_col)
        encoded_features.append(encoding_layer)
    else:
        input = tf.keras.Input(shape=(1,), name=i)
        encoded = tf.keras.layers.Dense(1, activation='relu')(input)
        all_inputs.append(input)
        encoded_features.append(encoded)

In [163]:
allfeatures = tf.keras.layers.concatenate(encoded_features)
model = tf.keras.models.Sequential([
    # tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(allfeatures),
    tf.keras.layers.Dense(512, activation='relu', input_shape=(x_train_reg.shape[1],), kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(allfeatures)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
column_model = tf.keras.Model(inputs=all_inputs, outputs=x)

In [164]:
column_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [165]:
column_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Wins (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 ORR (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 OFR (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 DFR (InputLayer)               [(None, 1)]          0           []                               
                                                                                            

In [151]:
x_train_reg.rename(columns={"Wins": "Wins", "OffensiveReboundRate": "ORR", "Seed": "SDF", "OffensiveRating": "OFR", "DefensiveRating": "DFR"}, inplace=True)
def df_to_dataset(dataframe, shuffle=True, batch_size=4000):
    dataframe = dataframe.copy()
    labels = y_train_cleaned
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

train_ds = df_to_dataset(x_train_reg, batch_size=4000)
[(train_feature_batch, label_batch)] = train_ds.take(1)

In [161]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = tf.keras.layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a layer that turns strings into integer indices.
  if dtype == 'string':
    index = tf.keras.layers.StringLookup(max_tokens=max_tokens)
  # Otherwise, create a layer that turns integer values into integer indices.
  else:
    index = tf.keras.layers.IntegerLookup(max_tokens=max_tokens)

  # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Encode the integer indices.
  encoder = tf.keras.layers.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply multi-hot encoding to the indices. The lambda function captures the
  # layer, so you can use them, or include them in the Keras Functional model later.
  return lambda feature: encoder(index(feature))
all_inputs = []
encoded_features = []
for i in ["Wins", "ORR", "SDF", "OFR", "DFR"]:
    if i == "SDF":
        categorical_encoding_layer = tf.keras.Input(shape=(1,), name=i, dtype='int64')
        encoding_layer = get_category_encoding_layer(i, train_ds, dtype='int64', max_tokens=16)
        encoded_categorical_col = encoding_layer(categorical_encoding_layer)
        all_inputs.append(categorical_encoding_layer)
        encoded = encoding_layer(input)
    else:
        numeric_col = tf.keras.Input(shape=(1,), name=i)
        normalization_layer = get_normalization_layer(i, train_ds)
        encoded_numeric_col = normalization_layer(numeric_col)
        all_inputs.append(numeric_col)
        encoded_features.append(encoded_numeric_col)

train_ds

2023-03-12 16:33:26.821846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-12 16:33:26.834433: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-12 16:33:27.004620: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-12 16:33:27.014074: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-12 16:33:27.173619: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-12 16:33:27.417635: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-12 16:33:27.432937: I tensorflow/core/grappler/optimizers/cust

<BatchDataset element_spec=({'Wins': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'ORR': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'SDF': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'OFR': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'DFR': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}, TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))>

In [167]:
column_model.fit(train_ds, epochs=100)

Epoch 1/100
22/23 [===========================>..] - ETA: 0s - loss: nan - accuracy: 0.5006

ValueError: in user code:

    File "/Users/Tim/mambaforge/envs/env_arm/lib/python3.9/site-packages/keras/engine/training.py", line 1557, in test_function  *
        return step_function(self, iterator)
    File "/Users/Tim/mambaforge/envs/env_arm/lib/python3.9/site-packages/keras/engine/training.py", line 1546, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/Tim/mambaforge/envs/env_arm/lib/python3.9/site-packages/keras/engine/training.py", line 1535, in run_step  **
        outputs = model.test_step(data)
    File "/Users/Tim/mambaforge/envs/env_arm/lib/python3.9/site-packages/keras/engine/training.py", line 1499, in test_step
        y_pred = self(x, training=False)
    File "/Users/Tim/mambaforge/envs/env_arm/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/Tim/mambaforge/envs/env_arm/lib/python3.9/site-packages/ke

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(x_train_reg, y_train_reg, epochs=1000, batch_size=16000, validation_data=(x_valid, y_valid))

Epoch 1/1000


2023-03-12 16:46:07.614763: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 2s 163ms/step - loss: nan - accuracy: 0.4994 - val_loss: nan - val_accuracy: 0.4990
Epoch 2/1000
1/6 [====>.........................] - ETA: 0s - loss: nan - accuracy: 0.4966

2023-03-12 16:46:08.806671: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 0s 61ms/step - loss: nan - accuracy: 0.4990 - val_loss: nan - val_accuracy: 0.4990
Epoch 3/1000
6/6 [==============================] - 0s 48ms/step - loss: nan - accuracy: 0.4997 - val_loss: nan - val_accuracy: 0.5010
Epoch 4/1000
6/6 [==============================] - 0s 49ms/step - loss: nan - accuracy: 0.5009 - val_loss: nan - val_accuracy: 0.5010
Epoch 5/1000
6/6 [==============================] - 0s 50ms/step - loss: nan - accuracy: 0.5006 - val_loss: nan - val_accuracy: 0.5010
Epoch 6/1000
6/6 [==============================] - 0s 47ms/step - loss: nan - accuracy: 0.5006 - val_loss: nan - val_accuracy: 0.5010
Epoch 7/1000
6/6 [==============================] - 0s 52ms/step - loss: nan - accuracy: 0.5006 - val_loss: nan - val_accuracy: 0.5010
Epoch 8/1000
6/6 [==============================] - 0s 46ms/step - loss: nan - accuracy: 0.5006 - val_loss: nan - val_accuracy: 0.5010
Epoch 9/1000
6/6 [==============================] - 0s 47ms/step - l

In [ ]:
model.evaluate(x_valid, y_valid)

In [123]:
y_valid.shape

(9333, 1)